In [1]:
import os
import pandas as pd
from distances import *
import numpy as np

In [2]:
def reader(path):
    with open(path, 'r') as f:
        text = f.read()
    return text

In [3]:
def distance(cwd, path_groundtrue, path_text):

    groundtrue_files = os.listdir(os.path.join(cwd, path_groundtrue))

    mean_CER = 0
    mean_WER = 0
    mean_jaccard = 0

    for file in groundtrue_files:
    
        t1 = reader(os.path.join(cwd,path_groundtrue,file))
        t2 = reader(os.path.join(cwd, path_text, file))

        _ , CER = levenshtein(t1,t2,'CER')
        _ , WER = levenshtein(t1,t2,'WER')
        mean_WER += WER
        mean_CER += CER
        print(file)
        print('\nWER levensthein: ',round(WER,2), '%\nCER levenshtein:',round(CER,2),'%')

        indice_jaccard = jaccard(t1,t2)
        ref_jaccard = jaccard(t1,t1)

    print('indice de Jaccard : ', round(indice_jaccard,4),'/',round(ref_jaccard,4))
    print('\nMoyenne WER: ',round(mean_WER/3,2))
    print('Moyenne CER: ',round(mean_CER/3,2))

In [6]:
cwd = '/home/lf/Bureau/Memoire/Corpus'

os.chdir(cwd)
print("Main ->",os.getcwd())
authors = os.listdir(cwd)

# dataframe pour chaque page de chaque ouvrage
df_page = pd.DataFrame(columns = ['Auteur', 'Ouvrage','Page','WER_Gallica','WER_Tesseract', 'WER_out','CER_Gallica','CER_Tesseract', 'CER_out','JAC_Ref','JAC_Gallica','JAC_Tesseract', 'JAC_out']) 
# dataframe pour la moyenne par ouvrage
df_ouvrage = pd.DataFrame(columns = ['Auteur', 'Ouvrage','WER_Gallica','WER_Tesseract', 'WER_out','CER_Gallica','CER_Tesseract', 'CER_out','JAC_Ref','JAC_Gallica','JAC_Tesseract', 'JAC_out']) 

for author in authors:

    ouvrages = os.listdir(author)

    for ouvrage in ouvrages:
        
        gallica = False
        tesseract = False
        out = False
        # déclarer le chemin vers les dossiers dont il y a un même texte issu de différent ocr à comparer avec le groundtrue
        path_groundtrue_dir = os.path.join(cwd, author, ouvrage, "TXT_GROUNDTRUE")
        path_tesseract_dir = os.path.join(cwd, author, ouvrage, "TXT_TESSERACT")
        path_gallica_dir = os.path.join(cwd, author, ouvrage, "TXT_GALLICA")
        path_out_dir = os.path.join(cwd, author, ouvrage, "TXT_out")
        
        if os.path.isdir(path_groundtrue_dir):
            
            groundtrue_files = sorted(os.listdir(path_groundtrue_dir))
            mean_WER2 = 0
            mean_WER3 = 0
            mean_WER4 = 0
            mean_CER2 = 0
            mean_CER3 = 0
            mean_CER4 = 0
            mean_JAC1 = 0
            mean_JAC2 = 0
            mean_JAC3 = 0
            mean_JAC4 = 0
            
            if os.path.isdir(path_gallica_dir):
                gallica = True
            
            if os.path.isdir(path_tesseract_dir):
                tesseract = True
            
            if os.path.isdir(path_out_dir):
                out = True
            
            for file_name in groundtrue_files:
                WER2,WER3,WER4 = np.nan, np.nan, np.nan
                
                t1 = reader(os.path.join(cwd,path_groundtrue_dir,file_name))
                JAC1 = jaccard(t1,t1)
                mean_JAC1 += JAC1
                
                if gallica:
                    t2 = reader(os.path.join(cwd,path_gallica_dir,file_name))
                    _ , CER2 = levenshtein(t1,t2,'CER')
                    mean_CER2 += CER2
                    _ , WER2 = levenshtein(t1,t2,'WER')
                    mean_WER2 += WER2
                    JAC2 = jaccard(t1,t2)
                    mean_JAC2 += JAC2

                if tesseract:
                    t3 = reader(os.path.join(cwd,path_tesseract_dir,file_name))
                    _ , CER3 = levenshtein(t1,t3,'CER')
                    mean_CER3 += CER3
                    _ , WER3 = levenshtein(t1,t3,'WER')
                    mean_WER3 += WER3
                    JAC3 = jaccard(t1,t3)
                    mean_JAC3 += JAC3
                if out:
                    t4 = reader(os.path.join(cwd,path_out_dir,file_name))
                    _ , CER4 = levenshtein(t1,t4,'CER')
                    mean_CER4 += CER4
                    _ , WER4 = levenshtein(t1,t4,'WER')
                    mean_WER4 += WER4
                    JAC4 = jaccard(t1,t4)
                    mean_JAC4 += JAC4

                a_series_page = pd.Series([author,ouvrage,file_name[:-4],round(WER2,2),round(WER3,2),round(WER4,2),round(CER2,2),round(CER3,2),round(CER4,2),round(JAC1,4),round(JAC2,4),round(JAC3,4),round(JAC4,4)], index = df_page.columns)
                df_page = df_page.append(a_series_page, ignore_index=True)
                
            a_series_ouvrage = pd.Series([author,ouvrage,round(mean_WER2/3,2),round(mean_WER3/3,2),round(mean_WER4/3,2),round(mean_CER2/3,2),round(mean_CER3/3,2),round(mean_CER4/3,2),round(mean_JAC1/3,4),round(mean_JAC2/3,4),round(mean_JAC3/3,4),round(mean_JAC4/3,4)], index = df_ouvrage.columns)
            df_ouvrage = df_ouvrage.append(a_series_ouvrage, ignore_index=True)
                
df_page.to_csv('/home/lf/Bureau/Memoire/Etapes/Corpus_ocr_distances_pages.txt')
df_ouvrage.to_csv('/home/lf/Bureau/Memoire/Etapes/Corpus_ocr_distances.txt')

Main -> /home/lf/Bureau/Memoire/Corpus
